

# Overview of the skore project

This example provides an overview of the functionalities and the different types
of items that we can store in a skore :class:`~skore.Project`.


## Creating and loading the skore project



We start by creating a temporary directory to store our project so that we can
easily clean it after executing this example:



In [ ]:
import tempfile
from pathlib import Path

temp_dir = tempfile.TemporaryDirectory(prefix="skore_example_")
temp_dir_path = Path(temp_dir.name)

We create and load the skore project from this temporary directory:



In [ ]:
import skore

my_project = skore.create("my_project", working_dir=temp_dir_path)

## Storing integers

Now, let us store our first object using :func:`~skore.Project.put`, for example an
integer:



In [ ]:
my_project.put("my_int", 3)

Here, the name of the object is ``my_int`` and the integer value is 3.

We can read it from the project by using :func:`~skore.Project.get`:



In [ ]:
my_project.get("my_int")

Careful; like in a traditional Python dictionary, the ``put`` method will *overwrite*
past data if we use a key that already exists!



In [ ]:
my_project.put("my_int", 30_000)

We can check the updated value:



In [ ]:
my_project.get("my_int")

.. seealso::

  Skore does not exactly *overwrite*, but stores the history of items.
  For more information about the tracking of items using their history,
  see `example_historization_items`.



By using the :func:`~skore.Project.delete_item` method, we can also delete an object
so that our skore UI does not become cluttered:



In [ ]:
my_project.put("my_int_2", 10)

In [ ]:
my_project.delete_item("my_int_2")

We can display all the keys in our project:



In [ ]:
my_project.list_item_keys()

## Storing strings and texts



We just stored a integer, now let us store some text using strings!



In [ ]:
my_project.put("my_string", "Hello world!")

In [ ]:
my_project.get("my_string")

:func:`~skore.Project.get` infers the type of the inserted object by default. For
example, strings are assumed to be in Markdown format. Hence, we can customize the
display of our text:



In [ ]:
my_project.put(
    "my_string_2",
    (
        """Hello world!, **bold**, *italic*, `code`

```python
def my_func(x):
    return x+2
```
"""
    ),
)

Moreover, we can also explicitly tell skore the media type of an object, for example
in HTML:



In [ ]:
from skore.item import MediaItem

my_project.put_item(
    "my_string_3",
    MediaItem.factory(
        "<p><h1>Title</h1> <b>bold</b>, <i>italic</i>, etc.</p>", media_type="text/html"
    ),
)

<div class="alert alert-info"><h4>Note</h4><p>We used :func:`~skore.Project.put_item` instead of :func:`~skore.Project.put`.</p></div>



Note that the media type is only used for the UI, and not in this notebook at hand:



In [ ]:
my_project.get("my_string_3")

We can also conveniently use a Python f-string:



In [ ]:
x = 2
y = [1, 2, 3, 4]
my_project.put("my_string_4", f"The value of `x` is {x} and the value of `y` is {y}.")

## Storing many kinds of data



Python list:



In [ ]:
my_list = [1, 2, 3, 4]
my_project.put("my_list", my_list)
my_list

Python dictionary:



In [ ]:
my_dict = {
    "company": "probabl",
    "year": 2023,
}
my_project.put("my_dict", my_dict)
my_dict

Numpy array:



In [ ]:
import numpy as np

my_arr = np.random.randn(3, 3)
my_project.put("my_arr", my_arr)
my_arr

Pandas data frame:



In [ ]:
import pandas as pd

my_df_pandas = pd.DataFrame(np.random.randn(10, 5))
my_project.put("my_df_pandas", my_df_pandas)
my_df_pandas.head()

Polars data frame:



In [ ]:
import polars as pl

my_df_polars = pl.DataFrame(np.random.randn(10, 5))
my_project.put("my_df_polars", my_df_polars)
my_df_polars.head()

## Storing data visualizations

Note that, in the dashboard, the interactivity of plots is supported, for example for
Altair and Plotly.



Matplotlib figure:



In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(0, 2, 100)

fig, ax = plt.subplots(layout="constrained", dpi=200)
ax.plot(x, x, label="linear")
ax.plot(x, x**2, label="quadratic")
ax.plot(x, x**3, label="cubic")
ax.set_xlabel("x label")
ax.set_ylabel("y label")
ax.set_title("Simple Plot")
ax.legend()
plt.show()

my_project.put("my_figure", fig)

|
Altair chart:



In [ ]:
import altair as alt

alt.renderers.enable("default")

num_points = 100
df_plot = pd.DataFrame(
    {"x": np.random.randn(num_points), "y": np.random.randn(num_points)}
)

my_altair_chart = (
    alt.Chart(df_plot)
    .mark_circle()
    .encode(x="x", y="y", tooltip=["x", "y"])
    .interactive()
    .properties(title="My title")
)

my_project.put("my_altair_chart", my_altair_chart)

<div class="alert alert-info"><h4>Note</h4><p>For Plotly figures, some users reported the following error when running Plotly
    cells: ``ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not
    installed``. This is a Plotly issue which is documented [here](https://github.com/plotly/plotly.py/issues/3285); to solve it, we recommend
    installing ``nbformat`` in your environment, e.g. with:

```console
pip install --upgrade nbformat</p></div>
```


Plotly figure:



In [ ]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter(
    df, x=df.sepal_length, y=df.sepal_width, color=df.species, size=df.petal_length
)

my_project.put("my_plotly_fig", fig)

fig

Animated Plotly figure:



In [ ]:
df = px.data.gapminder()
my_anim_plotly_fig = px.scatter(
    df,
    x="gdpPercap",
    y="lifeExp",
    animation_frame="year",
    animation_group="country",
    size="pop",
    color="continent",
    hover_name="country",
    log_x=True,
    size_max=55,
    range_x=[100, 100_000],
    range_y=[25, 90],
)

my_project.put("my_anim_plotly_fig", my_anim_plotly_fig)

my_anim_plotly_fig

PIL image:



In [ ]:
import io

import PIL

my_pil_image = PIL.Image.new("RGB", (100, 100), color="red")
with io.BytesIO() as output:
    my_pil_image.save(output, format="png")

my_project.put("my_pil_image", my_pil_image)

## Storing scikit-learn models and pipelines

As skore is developed by [Probabl](https://probabl.ai), the spin-off of
scikit-learn, skore treats scikit-learn models and pipelines as first-class citizens.

First of all, we can store a scikit-learn model:



In [ ]:
from sklearn.linear_model import Lasso

my_model = Lasso(alpha=2)
my_project.put("my_model", my_model)
my_model

We can also store scikit-learn pipelines:



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

my_pipeline = Pipeline(
    [("standard_scaler", StandardScaler()), ("lasso", Lasso(alpha=2))]
)
my_project.put("my_pipeline", my_pipeline)
my_pipeline

Moreover, we can store fitted scikit-learn pipelines:



In [ ]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X = diabetes.data[:150]
y = diabetes.target[:150]
my_pipeline.fit(X, y)

my_project.put("my_fitted_pipeline", my_pipeline)
my_pipeline

### Cleanup the project

Removing the temporary directory:



In [ ]:
temp_dir.cleanup()